In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
from scipy.sparse import hstack

In [3]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
df= pd.read_csv('parks_busyness_weather_normalized.csv')

C:\Users\Justh\AppData\Local\Temp\ipykernel_17672\3410809604.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('parks_busyness_weather_normalized.csv')


In [7]:
df.head()

,park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness,taxi_zone_ids,taxi_zones,normalized_park_busyness,park_busyness_percentile,geometry,datetime,dt_iso,temp,temp_min,temp_max,rain_1h,rain_3h
0,PK101,Inwood Hill Park,784904.717278,2022,1,1,11,5,1,4,1,3,2.104281e+06,1.319376e+06,0.373004,1.119011,243,Washington Heights North,1.425665e-06,15.893746,"POLYGON ((-73.9320015 40.86963009999998, -73.9...",2022-01-01 11:00:00,2022-01-01 11:00:00,10.21,9.22,11.00,0.25,0.0
1,PK101,Inwood Hill Park,784904.717278,2022,1,1,17,5,1,4,1,6,2.104281e+06,1.319376e+06,0.373004,2.238022,243,Washington Heights North,2.851330e-06,30.453822,"POLYGON ((-73.9320015 40.86963009999998, -73.9...",2022-01-01 17:00:00,2022-01-01 17:00:00,12.12,11.47,12.78,0.38,0.0
2,PK101,Inwood Hill Park,784904.717278,2022,1,1,19,5,1,4,1,6,2.104281e+06,1.319376e+06,0.373004,2.238022,243,Washington Heights North,2.851330e-06,30.453822,"POLYGON ((-73.9320015 40.86963009999998, -73.9...",2022-01-01 19:00:00,2022-01-01 19:00:00,12.16,11.45,12.81,0.25,0.0
3,PK101,Inwood Hill Park,784904.717278,2022,1,10,12,0,0,4,0,1,2.104281e+06,1.319376e+06,0.373004,0.373004,243,Washington Heights North,4.752217e-07,3.029150,"POLYGON ((-73.9320015 40.86963009999998, -73.9...",2022-01-10 12:00:00,2022-01-10 12:00:00,-1.61,-2.21,-0.78,0.00,0.0
4,PK101,Inwood Hill Park,784904.717278,2022,1,10,13,0,0,4,0,1,2.104281e+06,1.319376e+06,0.373004,0.373004,243,Washington Heights North,4.752217e-07,3.029150,"POLYGON ((-73.9320015 40.86963009999998, -73.9...",2022-01-10 13:00:00,2022-01-10 13:00:00,-1.88,-3.21,-1.22,0.00,0.0


In [8]:
df.dtypes

park_id                      object
park_name                    object
park_area                   float64
year                          int64
month                         int64
day                           int64
hour                          int64
day_of_week                   int64
weekend                       int64
season                        int64
isHoliday?                    int64
taxi_zone_busyness            int64
taxi_zone_area              float64
remaining_taxi_zone_area    float64
park_proportion             float64
park_busyness               float64
taxi_zone_ids                object
taxi_zones                   object
normalized_park_busyness    float64
park_busyness_percentile    float64
geometry                     object
datetime                     object
dt_iso                       object
temp                        float64
temp_min                    float64
temp_max                    float64
rain_1h                     float64
rain_3h                     

In [9]:
categorical_features= ['park_id', 'day_of_week','weekend','season','isHoliday?']

In [10]:
df[categorical_features]=df[categorical_features].astype(str)

In [11]:
df.dtypes

park_id                      object
park_name                    object
park_area                   float64
year                          int64
month                         int64
day                           int64
hour                          int64
day_of_week                  object
weekend                      object
season                       object
isHoliday?                   object
taxi_zone_busyness            int64
taxi_zone_area              float64
remaining_taxi_zone_area    float64
park_proportion             float64
park_busyness               float64
taxi_zone_ids                object
taxi_zones                   object
normalized_park_busyness    float64
park_busyness_percentile    float64
geometry                     object
datetime                     object
dt_iso                       object
temp                        float64
temp_min                    float64
temp_max                    float64
rain_1h                     float64
rain_3h                     

In [12]:
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[categorical_features]) 

In [13]:
numeric_features= ['temp','rain_1h','rain_3h']

In [14]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[numeric_features])

In [15]:
if encoded_features.shape[0] == scaled_features.shape[0]:
    scaled_features_s= csr_matrix(scaled_features)
    


In [16]:
encoded_features.shape

(336365, 206)

In [17]:
scaled_features.shape

(336365, 3)

In [18]:
combined_features = hstack((encoded_features, scaled_features))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(combined_features, df['park_busyness_percentile'], test_size=0.2, random_state=42)

In [22]:
# Train a Random Forest model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)



In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")